## Dependencies
* Firrtl [Installation Instructions](https://github.com/freechipsproject/firrtl#installation-instructions)   
  Add the directory containing the firrtl command line tool (`firrtl/utils/bin`) to your `$PATH`

In [1]:
from magma import *
main = DefineCircuit('main', "a", In(Bit), "b", In(Bit), "c", In(Bit), "d", Out(Bit))
d = (main.a & main.b) ^ main.c
wire(d, main.d)
compile("main", main, output="firrtl")

with open("main.fir", "r") as f:
    print(f.read())

compiling main
circuit main :
  module main :
    input a : UInt<1>
    input b : UInt<1>
    input c : UInt<1>
    output d : UInt<1>
    
    wire inst0_out : UInt<1>
    wire inst1_out : UInt<1>
    inst0_out <= and(a, b)
    inst1_out <= xor(inst0_out, c)
    d <= inst1_out



*Note*: the `!` syntax used in the next cell is jupyter notebook syntax sugar for executing a shell command

In [2]:
!firrtl -i main.fir -o main.v -X verilog

Total FIRRTL Compile Time: 286.4 ms


In [3]:
with open("main.v", "r") as f:
    print(f.read())

`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif

module main(
  input   a,
  input   b,
  input   c,
  output  d
);
  wire  inst0_out;
  wire  inst1_out;
  assign d = inst1_out;
  assign inst0_out = a & b;
  assign inst1_out = inst0_out ^ c;
endmodule



In [4]:
with open("sim_main.cpp", "w") as sim_main_f:
    sim_main_f.write("""
#include "Vmain.h"
#include "verilated.h"
#include <cassert>
#include <iostream>

int main(int argc, char **argv, char **env) {
    Verilated::commandArgs(argc, argv);
    Vmain* top = new Vmain;
    int tests[8][4] = {
        {0, 0, 0, 0},
        {1, 0, 0, 0},
        {0, 1, 0, 0},
        {1, 1, 0, 1},
        {0, 0, 1, 1},
        {1, 0, 1, 1},
        {0, 1, 1, 1},
        {1, 1, 1, 0},
    };
    for(int i = 0; i < 8; i++) {
        int* test = tests[i];
        int a = test[0];
        int b = test[1];
        int c = test[2];
        int d = test[3];

        top->a = a;
        top->b = b;
        top->c = c;

        top->eval();
        assert(top->d == d);
    }

    delete top;
    std::cout << "Success" << std::endl;
    exit(0);
}    
""")

*Note*: The `%%bash` statement is a jupyter notebook magic operator that treats the cell as a bash script

In [5]:
%%bash
verilator -Wall -Wno-DECLFILENAME --cc main.v --exe sim_main.cpp
make -C obj_dir -j -f Vmain.mk Vmain
./obj_dir/Vmain

clang++  -I.  -MMD -I/usr/local/Cellar/verilator/3.900/share/verilator/include -I/usr/local/Cellar/verilator/3.900/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -Wno-char-subscripts -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-parameter -Wno-unused-variable -fbracket-depth=4096 -Qunused-arguments       -c -o sim_main.o ../sim_main.cpp
/usr/bin/perl /usr/local/Cellar/verilator/3.900/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include Vmain.cpp > Vmain__ALLcls.cpp
/usr/bin/perl /usr/local/Cellar/verilator/3.900/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include Vmain__Syms.cpp > Vmain__ALLsup.cpp
clang++  -I.  -MMD -I/usr/local/Cellar/verilator/3.900/share/verilator/include -I/usr/local/Cellar/verilator/3.900/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -Wno-char-subscripts -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-